# GameAnswerAgent Project

## The Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building GameAnswerAgent, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
import os
from dotenv import load_dotenv
from lib.tooling import tool
from tavily import TavilyClient
from datetime import datetime
from typing import Dict
import json

In [4]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")
CHROMA_OPENAI_API_BASE = os.getenv("CHROMA_OPENAI_API_BASE")

In [5]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="chromadb")
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("CHROMA_OPENAI_API_KEY"),
    model_name="text-embedding-3-small",
    api_base=os.getenv("CHROMA_OPENAI_API_BASE", "https://api.openai.com/v1")
)

collection = chroma_client.get_collection("udaplay")

client = TavilyClient(api_key=TAVILY_API_KEY)

### Tools

We are going to build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool
The `retrieve_game` tool is a semantic search function that queries a vector database for video game information based on a user's input. It uses the ChromaDB collection to find the most relevant results matching the query, leveraging embeddings for similarity search. The function returns a list of dictionaries, each containing key details about a game (name, platform, release year, description). This tool enables the agent to answer questions using structured, contextually relevant data from the internal database, making it suitable for retrieval-augmented generation (RAG) workflows.

In [6]:
@tool(
    name="retrieve_game",
    description="Semantic search: Finds most results in the vector DB"
)
def retrieve_game(query: str):
    """
    Retrieve game information based on a query.
    
    Args:
        query (str): The query to search for in the game database.
    
    Returns:
        list: A list of dictionaries containing game information.
    """
    
    # Use the chroma client and collection to perform the search
    results = collection.query(
        query_texts=query, n_results=1)
    
    formatted_results = []

    # results['metadatas'] is a list of lists → [[metadata1, metadata2, ...]]
    for metadata in results["metadatas"][0]:  # Access the first query’s results
        formatted_results.append({
            "Name": metadata.get("Name"),
            "Platform": metadata.get("Platform"),
            "YearOfRelease": metadata.get("YearOfRelease"),
            "Description": metadata.get("Description")
        })
    
    return formatted_results

#### Evaluate Retrieval Tool
The `evaluate_retrieval` tool is designed to assess the usefulness of documents retrieved from the vector database in answering a user's question. It leverages the `AgentEvaluator` class, which provides automated evaluation capabilities using an LLM. The tool parses the retrieved documents, formats them for clarity, and creates a test case describing the user's query and the expected outcome. The `AgentEvaluator` then analyzes the agent's response (the retrieved documents) and generates feedback on whether the information is sufficient and relevant. This process ensures that the agent only relies on internal knowledge when it is adequate, and can trigger a web search if the evaluation finds gaps. The use of `AgentEvaluator` enables objective, consistent, and scalable evaluation of retrieval quality within the agent workflow.

In [7]:
from lib.evaluation import AgentEvaluator, TestCase
from json_repair import repair_json

@tool(
    name="evaluate_retrieval",
    description="Based on the user's question and on the list of retrieved documents, it will analyze the usability of the documents to respond to that question."
)
def evaluate_retrieval(question: str, retrieved_docs: str):
    """
    Evaluate the relevance of retrieved documents for a given question.

    Args:
        question (str): The original question from the user.
        retrieved_docs (list): The list of retrieved documents.

    Returns:
        dict: A report on the usefulness of the retrieved documents.
    """
    # Use the AgentEvaluator class for document retrieval evaluation
    evaluator = AgentEvaluator(api_key=OPENAI_API_KEY, base_url=OPENAI_API_BASE)
        
    try:
        parsed_docs = json.loads(repair_json(retrieved_docs))
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON passed to evaluate_retrieval: {e}. Original input: {retrieved_docs} ")
    
    # Format the retrieved documents for evaluation
    docs_text = ""
    if parsed_docs:
        for i, doc in enumerate(parsed_docs, start=1):
            if isinstance(doc, dict):
                # Handle structured document format
                platform = doc.get('Platform', 'Unknown')
                name = doc.get('Name', 'Unknown')
                year = doc.get('YearOfRelease', 'Unknown')
                description = doc.get('Description', 'No description')
                docs_text += f"Document {i}:\n- Platform: {platform}\n- Name: {name}\n- Year: {year}\n- Description: {description}\n\n"
            else:
                # Handle plain text documents
                docs_text += f"Document {i}: {str(doc)}\n\n"
    else:
        docs_text = "No documents retrieved."
    
    # Create a mock TestCase for document evaluation
    test_case = TestCase(
        id="doc_eval",
        description="Evaluate if retrieved documents are sufficient to answer the user's question",
        user_query=question,
        expected_tools=["retrieve_game"],
        reference_answer="Documents should contain relevant information to answer the question"
    )
    
    # Use evaluate_final_response with the formatted documents as the agent response
    evaluation_result = evaluator.evaluate_final_response(
        test_case=test_case,
        agent_response=docs_text,
        execution_time=0.0,
        total_tokens=0
    )
    
    # Extract the useful information and return in the expected format
    return {
        "useful": evaluation_result.task_completion.task_completed,
        "description": evaluation_result.feedback
    }

#### Game Web Search Tool

The `game_web_search` tool enables the agent to search the web for video game information using the Tavily API. When invoked, it takes a user's query and an optional search depth parameter (`basic` or `advanced`). The tool sends the query to Tavily, requesting a concise answer and relevant search results. It returns a structured dictionary containing the answer, a list of results, and metadata such as the timestamp and original query. This tool is used when internal knowledge is insufficient, allowing the agent to provide up-to-date information from external sources.


In [8]:
@tool
def game_web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    client = TavilyClient(api_key=TAVILY_API_KEY)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

### Agent
#### GameAnswerAgent Architecture

##### **State Machine Workflow:**
1. **UserQuery** → **RAG Step** → **Evaluation** → **Decision Point**
2. If evaluation shows RAG results are sufficient → **Answer Step** → **Terminate**
3. If evaluation shows RAG results are insufficient → **Web Step** → **Answer Step** → **Terminate**

##### **Key Components:**

1. **GameAgentState**: Custom state schema that tracks:
   - `user_query`: The original question
   - `rag_results`: Retrieved documents from vector DB
   - `evaluation_result`: Assessment of document relevance
   - `web_results`: Web search results (if needed)
   - `final_answer`: The composed final response
   - `needs_web_search`: Decision flag

2. **Sub-Agents**: Exactly as you specified:
   ```python
   # RAG agent with retrieval + evaluation tools
   self.rag_agent = Agent(
       model_name=model_name,
       instructions=rag_instructions,
       tools=[retrieve_game, evaluate_retrieval],
   )

   # Web agent with search tool
   self.web_agent = Agent(
       model_name=model_name,
       instructions=web_instructions,
       tools=[game_web_search],
   )
   ```

3. **Workflow Steps**:
   - **`_rag_step`**: Invokes RAG agent to retrieve docs and evaluate them
   - **`_web_step`**: Invokes Web agent when RAG results are insufficient
   - **`_answer_step`**: Combines available information into final answer

4. **Smart Transitions**: Uses conditional logic to decide whether web search is needed based on the evaluation result

## Running the agent

In [9]:
from lib.game_answer_agent import GameAnswerAgent# Create the GameAnswerAgent

game_agent = GameAnswerAgent(
    model_name="gpt-4o-mini", 
    retrieve_game=retrieve_game,
    evaluate_retrieval=evaluate_retrieval,
    game_web_search=game_web_search,
    base_url=OPENAI_API_BASE, 
    api_key=OPENAI_API_KEY,
    log_level="debug")

[DEBUG] Handlers on logger: 1


In [10]:
answer = game_agent.ask("Was Mortal Kombat X released for Playstation 5?")
print()
print(f"Final answer: {answer}")

[INFO] Starting workflow for query: 'Was Mortal Kombat X released for Playstation 5?'
[DEBUG] [StateMachine] Starting: __entry__
[DEBUG] RAG Step: Processing query 'Was Mortal Kombat X released for Playstation 5?'
[DEBUG] [StateMachine] Starting: __entry__
[DEBUG] [StateMachine] Executing step: message_prep
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Terminating: __termination__
[DEBUG] RAG results: [{'Name': 'Halo Infinite', 'Platform': 'Xbox Series X|S', 'YearOfRelease': 2021, 'Description': "The latest installment in the Halo franchise


Final answer: Based on web search:
Mortal Kombat X was originally released for PlayStation 4 on April 14, 2015. While it is not a dedicated PlayStation 5 title, it is playable on the PS5 console. However, some features available on the PS4 version may be absent when played on the PS5, and online features require an account to access.

For more details, you can check the [Wikipedia page for Mortal Kombat X](https://en.wikipedia.org/wiki/Mortal_Kombat_X) or the [PlayStation page](https://www.playstation.com/en-us/games/mortal-kombat-x_msm_moved/).




The implementation properly uses the existing Agent class for the sub-components and leverages the state machine framework to create a sophisticated workflow that can intelligently decide when to fall back to web search based on the quality of retrieved documents.

## Multiple test cases

In [11]:
# Test the GameAnswerAgent with the three sample queries

print("=" * 80)
print("Testing GameAnswerAgent")
print("=" * 80)

# Test Query 1: When Pokémon Gold and Silver was released?
print("\n1. Testing: When Pokémon Gold and Silver was released?")
print("-" * 60)
answer1 = game_agent.ask("When Pokémon Gold and Silver was released?")
print(f"Answer: {answer1}")

print("\n" + "=" * 80)

# Test Query 2: Which one was the first 3D platformer Mario game?
print("\n2. Testing: Which one was the first 3D platformer Mario game?")
print("-" * 60)
answer2 = game_agent.ask("Which one was the first 3D platformer Mario game?")
print(f"Answer: {answer2}")

print("\n" + "=" * 80)

# Test Query 3: Was Mortal Kombat X released for Playstation 5?
print("\n3. Testing: Was Mortal Kombat X released for Playstation 5?")
print("-" * 60)
answer3 = game_agent.ask("Was Mortal Kombat X released for Playstation 5?")
print(f"Answer: {answer3}")

print("\n" + "=" * 80)
print("Testing Complete!")
print("=" * 80)

[INFO] Starting workflow for query: 'When Pokémon Gold and Silver was released?'
[DEBUG] [StateMachine] Starting: __entry__
[DEBUG] RAG Step: Processing query 'When Pokémon Gold and Silver was released?'
[DEBUG] [StateMachine] Starting: __entry__
[DEBUG] [StateMachine] Executing step: message_prep


Testing GameAnswerAgent

1. Testing: When Pokémon Gold and Silver was released?
------------------------------------------------------------


[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Terminating: __termination__
[DEBUG] RAG results: [{'Name': 'Pokémon Gold and Silver', 'Platform': 'Game Boy Color', 'YearOfRelease': 1999, 'Description': 'Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.'}]
[DEBUG] Evaluation result: {'useful': True, 'description': "The agent response successfully provides relevant information about the release of Pokémon Gold and Silver, including the year of release (1999), which directly answers the user's query. The format is clear and structured, listing the details in an organized manner. Additionally, the response adheres to the implicit instruction to include relevant information to answer the question."}
[DEBUG] [StateM

Answer: Based on our game database:
- Pokémon Gold and Silver (Game Boy Color, 1999): Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.



2. Testing: Which one was the first 3D platformer Mario game?
------------------------------------------------------------


[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Terminating: __termination__
[DEBUG] RAG results: [{'Name': 'Super Mario 64', 'Platform': 'Nintendo 64', 'YearOfRelease': 1996, 'Description': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."}]
[DEBUG] Evaluation result: {'useful': True, 'description': "The agent response successfully identifies 'Super Mario 64' as the first 3D platformer Mario game, which directly answers the user's query. The format is clear and structured, providing relevant details about the game. Additionally, the response adheres to the implicit instructions by including sufficient information to address the user's question."}
[DEBUG] [StateMachine] Executing 

Answer: Based on our game database:
- Super Mario 64 (Nintendo 64, 1996): A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.



3. Testing: Was Mortal Kombat X released for Playstation 5?
------------------------------------------------------------


[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Executing step: tool_executor
[DEBUG] [StateMachine] Executing step: llm_processor
[DEBUG] [StateMachine] Terminating: __termination__
[DEBUG] RAG results: [{'Name': 'Halo Infinite', 'Platform': 'Xbox Series X|S', 'YearOfRelease': 2021, 'Description': "The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting."}]
[DEBUG] Evaluation result: {'useful': False, 'description': "The agent response did not address the user's query about Mortal Kombat X and instead provided information about a different game, Halo Infinite. Therefore, it did not complete the t

Answer: Based on web search:
Mortal Kombat X was not released as a dedicated title for PlayStation 5. It was originally released for PlayStation 4 on April 14, 2015. However, it is playable on the PS5, although some features available in the PS4 version may not be present, and online features require an account to use.

Testing Complete!


### Proving conversational state

In [ ]:
from framework.game_answer_agent import GameAnswerAgent# Create the GameAnswerAgent

game_agent = GameAnswerAgent(
    model_name="gpt-4o-mini", 
    retrieve_game=retrieve_game,
    evaluate_retrieval=evaluate_retrieval,
    game_web_search=game_web_search,
    base_url=OPENAI_API_BASE, 
    api_key=OPENAI_API_KEY,
    log_level="debug",
    session_id="s1")

In [ ]:
print("=" * 80)
print("Testing GameAnswerAgent's conversational state")
print("=" * 80)

# Test Query 1: When Pokémon Gold and Silver was released?
print("\n1. Testing: When Pokémon Gold and Silver was released?")
print("-" * 60)
answer1 = game_agent.ask("When Pokémon Gold and Silver was released?")
print(f"Answer: {answer1}")

print("\n" + "=" * 80)

print("\n2. Testing: Which was the game I asked you about?")
print("-" * 60)
answer2 = game_agent.ask("Which was the game I asked you about?")
print(f"Answer: {answer2}")

print("\n" + "=" * 80)

print("\n2. Testing: Describe the main features of the game I asked you about?")
print("-" * 60)
answer3 = game_agent.ask("What are the main features of the game I asked you about?")
print(f"Answer: {answer3}")

## Results Report: GameAnswerAgent Evaluation

### Query 1: When Pokémon Gold and Silver was released?
**Agent Response:**  
Based on our game database:  
- Pokémon Gold and Silver (Game Boy Color, 1999): Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.

**Assessment:**  
The agent successfully retrieved accurate information from the internal database, including platform and release year.

---

### Query 2: Which one was the first 3D platformer Mario game?
**Agent Response:**  
Based on our game database:  
- Super Mario 64 (Nintendo 64, 1996): A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.

**Assessment:**  
The agent correctly identified Super Mario 64 as the first 3D Mario platformer, providing relevant details from the database.

---

### Query 3: Was Mortal Kombat X released for Playstation 5?
**Agent Response:**  
Based on web search:  
Mortal Kombat X was not released as a dedicated title for PlayStation 5. It was originally released for PlayStation 4 on April 14, 2015. However, it is playable on the PS5, although some features available in the PS4 version may not be present, and online features require an account to use.

**Assessment:**  
The agent determined that internal data was insufficient and correctly utilized web search to provide a comprehensive answer, clarifying compatibility and release details.

---

### Summary

- The GameAnswerAgent accurately answers questions using both internal knowledge and web search when necessary.
- For database-covered queries, responses are concise and relevant.
- For queries beyond the database scope, the agent seamlessly transitions to web search, ensuring complete and up-to-date information.